# benchmark: medpy vs. mikan-rs 🍊

Please note that **Colab only provides dual-core CPUs**, so the speedup is limited. You can test on a server with more CPUs to observe mikan's blazingly fast performance.

In [1]:
!pip install SimpleITK medpy mikan-rs

In [2]:
import gdown

# We use datasets from seg_metrics

gdth_url_id = "1R6rph1_Wc2HfLhzvGkcNt7hgbQiHWZBY"
pred_url_id = "1cShgX96WgK_j4EbfR4wN2RLgcyVR4ca0"
gdth_fpath = "patients_26_ground_truth.nii.gz"
pred_fpath = "patients_26_segmentation.nii.gz"

gdown.download(id=gdth_url_id, output=gdth_fpath, quiet=False)
gdown.download(id=pred_url_id, output=pred_fpath, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1R6rph1_Wc2HfLhzvGkcNt7hgbQiHWZBY
To: /content/patients_26_ground_truth.nii.gz
100%|██████████| 1.17M/1.17M [00:00<00:00, 70.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1cShgX96WgK_j4EbfR4wN2RLgcyVR4ca0
To: /content/patients_26_segmentation.nii.gz
100%|██████████| 5.05M/5.05M [00:00<00:00, 28.3MB/s]


'patients_26_segmentation.nii.gz'

In [2]:
import time
import mikan
import numpy as np
import SimpleITK as sitk
from medpy.metric import dc, hd, hd95, assd

## Load datas

In [ ]:
gt = sitk.ReadImage(rf"patients_26_ground_truth.nii.gz", sitk.sitkUInt8)
pred = sitk.ReadImage(rf"patients_26_segmentation.nii.gz", sitk.sitkUInt8)

gt_arr = sitk.GetArrayFromImage(gt)
pred_arr = sitk.GetArrayFromImage(pred)

# Downsample for faster
# If you're patient, you can comment out here and wait for medpy to run for 30 minutes 😆
gt_arr = np.array(gt_arr[::2, ::2, ::2])
pred_arr = np.array(pred_arr[::2, ::2, ::2])

## Dice

In [13]:
# mikan: DSC
t = time.time()
evaluator = mikan.ArrayEvaluator(gt_arr, pred_arr, spacing=gt.GetSpacing())
dsc = evaluator.labels([1,2,3,4,5]).metrics("dsc")
mikan_costs = time.time() - t
print(f"Mikan cost {mikan_costs:.2f} s.")

# medpy: DSC
t = time.time()
for i in (1,2,3,4,5):
    dsc = dc(pred_arr == i, gt_arr == i)
medpy_costs = time.time() - t
print(f"medpy costs {time.time() - t:.2f} s.")
print(f"DSC: {medpy_costs / mikan_costs :.2f}x faster")


Mikan cost 0.03 s.
medpy costs 0.27 s.
DSC: 8.05x faster


## HD

In [10]:
# mikan: HD
t = time.time()
evaluator = mikan.ArrayEvaluator(gt_arr, pred_arr, spacing=gt.GetSpacing())
hausdorff_distance = evaluator.labels([1, 2, 3, 4, 5]).metrics("HD")
print(hausdorff_distance)

mikan_costs = time.time() - t
print(f"Mikan has calculated Hausdorff distance and cost {mikan_costs:.2f} s.")
print(f"Let's waiting for medpy, be patient for a while...")

# medpy: HD
t = time.time()
for i in (1, 2, 3, 4, 5):
    hausdorff_distance = hd(pred_arr == i, gt_arr == i, voxelspacing=gt.GetSpacing()[::-1])
medpy_costs = time.time() - t
print(hausdorff_distance)
print(f"HD: {medpy_costs / mikan_costs :.2f}x faster")

[3.8066796490554484, 5.875891921349948, 7.18959419428273, 55.03783156368531, 46.23897571237574]
Mikan has calculated Hausdorff distance and cost 0.93 s.
Let's waiting for medpy, be patient for a while...
46.23897571237574
HD: 71.22x faster


## All Distances

In [9]:
# mikan: Distances
t = time.time()
evaluator = mikan.ArrayEvaluator(gt_arr, pred_arr, spacing=gt.GetSpacing())
mikan_distances = evaluator.labels([1,2,3,4,5]).metrics(["hd", "hd95", "assd"])
mikan_costs = time.time() - t
print(mikan_distances)
print(f"Mikan has calculated distance and cost {mikan_costs:.2f} s.")
print(f"Let's waiting for medpy, be patient for a while...")

# medpy: Distances
t = time.time()
medpy_results = {}
for i in (1,2,3,4,5):
    hd_ = hd(pred_arr == i, gt_arr == i, voxelspacing=gt.GetSpacing()[::-1])
    hd95_ = hd95(pred_arr == i, gt_arr == i, voxelspacing=gt.GetSpacing()[::-1])
    assd_ = assd(pred_arr == i, gt_arr == i, voxelspacing=gt.GetSpacing()[::-1])
    medpy_results[i] = {
        "hd": hd_,
        "hd95": hd95_,
        "assd": assd_,
    }
medpy_costs = time.time() - t
print(medpy_results)
print(f"Distances: {medpy_costs / mikan_costs :.2f}x faster")

{'1': {'hd': 3.8066796490554484, 'hd95': 0.7409999966621399, 'assd': 0.2482734556966591}, '2': {'hd': 5.875891921349948, 'hd95': 0.893913303991663, 'assd': 0.30188703831217684}, '3': {'hd': 7.18959419428273, 'hd95': 0.7409999966621399, 'assd': 0.29096373483885757}, '4': {'hd': 55.03783156368531, 'hd95': 0.7409999966621399, 'assd': 0.25367831066384994}, '5': {'hd': 46.23897571237574, 'hd95': 0.7409999966621399, 'assd': 0.27452537658032045}}
Mikan has calculated distance and cost 0.96 s.
Let's waiting for medpy, be patient for a while...
{1: {'hd': 3.8066796490554484, 'hd95': 0.7409999966621399, 'assd': 0.24827345569665876}, 2: {'hd': 5.875891921349948, 'hd95': 0.893913303991663, 'assd': 0.3018870383121832}, 3: {'hd': 7.18959419428273, 'hd95': 0.7409999966621399, 'assd': 0.2909637348388582}, 4: {'hd': 55.03783156368531, 'hd95': 0.7409999966621399, 'assd': 0.25367831066385055}, 5: {'hd': 46.23897571237574, 'hd95': 0.7409999966621399, 'assd': 0.2745253765803193}}
Distances: 210.35x faster
